In [1]:
## 1. Import Libraries
import pandas as pd
import numpy as np
import os
import csv
import gc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.models import Model

In [2]:
# ## 2. Load Datasets
def load_data_from_files(folder_path, columns):
    all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.txt')]
    dataframes = []
    for file in all_files:
        df = pd.read_csv(file, names=columns
                         , delimiter="\t", quoting=csv.QUOTE_NONE, encoding='utf-8')
        df['line_id'] = file.split('/')[-1].replace('.txt', '')
        dataframes.append(df)
    return pd.concat(dataframes, ignore_index=True)
    
train_data = load_data_from_files('C:\\Users\\ThugCom\\project\\superai_ss5\\super-ai-ss-5-named-entity-recognition\\train\\train'
                                  , ['word', 'pos', 'ner', 'clause_boundary'])
eval_data = load_data_from_files('C:\\Users\\ThugCom\\project\\superai_ss5\\super-ai-ss-5-named-entity-recognition\\eval\\eval'
                                 , ['word', 'pos', 'ner', 'clause_boundary'])
test_data = load_data_from_files('C:\\Users\\ThugCom\\project\\superai_ss5\\super-ai-ss-5-named-entity-recognition\\test\\test'
                                 , ['word', 'pos', 'clause_boundary'])

In [3]:
# ## 3. Preprocess Data

# Clean data
train_data['ner'] = train_data['ner'].apply(lambda x: 'O' if x == 'B' else x)
eval_data['ner'] = eval_data['ner'].apply(lambda x: 'O' if x == 'B' else x)

# Encode words, POS tags, and NER labels
word_encoder = LabelEncoder()
pos_encoder = LabelEncoder()
ner_encoder = LabelEncoder()
clause_encoder = LabelEncoder()

train_data['word'] = word_encoder.fit_transform(train_data['word'])
train_data['pos'] = pos_encoder.fit_transform(train_data['pos'])
train_data['ner'] = ner_encoder.fit_transform(train_data['ner'])
train_data['clause_boundary'] = clause_encoder.fit_transform(train_data['clause_boundary'])

# Create sequences for training
X_word = train_data.groupby('line_id')['word'].apply(list).values
X_pos = train_data.groupby('line_id')['pos'].apply(list).values
X_clause = train_data.groupby('line_id')['clause_boundary'].apply(list).values
Y_ner = train_data.groupby('line_id')['ner'].apply(list).values

del train_data
gc.collect()

# Pad sequences
# max_len = max([len(seq) for seq in X_word])
max_len = int(np.percentile([len(seq) for seq in X_word], 95))

X_word = pad_sequences(X_word, maxlen=max_len, padding='post')
X_pos = pad_sequences(X_pos, maxlen=max_len, padding='post')
X_clause = pad_sequences(X_clause, maxlen=max_len, padding='post')
Y_ner = pad_sequences(Y_ner, maxlen=max_len, padding='post')
# Y_ner = [to_categorical(i, num_classes=len(ner_encoder.classes_)) for i in Y_ner]
# Y_ner = np.array([to_categorical(i, num_classes=len(ner_encoder.classes_)) for i in Y_ner])

# Split train/eval dataset
X_train_word, X_val_word, X_train_pos, X_val_pos, X_train_clause, X_val_clause, Y_train, Y_val = train_test_split(
    X_word, X_pos, X_clause, Y_ner, test_size=0.2, random_state=42
)


In [4]:
print(f"Number of classes in ner_encoder: {len(ner_encoder.classes_)}")
print(f"Any NaN in Y_ner: {np.isnan(Y_ner).any()}")


Number of classes in ner_encoder: 40
Any NaN in Y_ner: False


In [5]:
# vocab_size ของ X_word
word_vocab_size = np.max(X_word) + 1
print(f"Vocab size for X_word: {word_vocab_size}")

# vocab_size ของ X_pos
pos_vocab_size = np.max(X_pos) + 1
print(f"Vocab size for X_pos: {pos_vocab_size}")

# vocab_size ของ X_clause
clause_vocab_size = np.max(X_clause) + 1
print(f"Vocab size for X_clause: {clause_vocab_size}")

print(f"Max value in X_word: {np.max(X_word)}, Min value: {np.min(X_word)}")
print(f"Max value in X_pos: {np.max(X_pos)}, Min value: {np.min(X_pos)}")
print(f"Max value in X_clause: {np.max(X_clause)}, Min value: {np.min(X_clause)}")

print(f"Max value in X_word: {np.max(X_word)}")
print(f"Max value in X_pos: {np.max(X_pos)}")
print(f"Max value in X_clause: {np.max(X_clause)}")

Vocab size for X_word: 28036
Vocab size for X_pos: 16
Vocab size for X_clause: 4
Max value in X_word: 28035, Min value: 0
Max value in X_pos: 15, Min value: 0
Max value in X_clause: 3, Min value: 0
Max value in X_word: 28035
Max value in X_pos: 15
Max value in X_clause: 3


In [6]:
# ## 4. Build and Compile the Model

input_word = Input(shape=(max_len,), name='word_input')
input_pos = Input(shape=(max_len,), name='pos_input')
input_clause = Input(shape=(max_len,), name='clause_input')

# embedding_word = Embedding(input_dim=len(word_encoder.classes_), output_dim=64, input_length=max_len)(input_word)
# embedding_pos = Embedding(input_dim=len(pos_encoder.classes_), output_dim=16, input_length=max_len)(input_pos)
# embedding_clause = Embedding(input_dim=2, output_dim=8, input_length=max_len)(input_clause)

embedding_word = Embedding(input_dim=word_vocab_size, output_dim=32, input_length=max_len)(input_word)
embedding_pos = Embedding(input_dim=pos_vocab_size, output_dim=8, input_length=max_len)(input_pos)
embedding_clause = Embedding(input_dim=clause_vocab_size, output_dim=4, input_length=max_len)(input_clause)

merged = tf.keras.layers.Concatenate()([embedding_word, embedding_pos, embedding_clause])
lstm_layer = Bidirectional(LSTM(units=64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))(merged)
output = TimeDistributed(Dense(len(ner_encoder.classes_), activation='softmax'))(lstm_layer)

model = Model(inputs=[input_word, input_pos, input_clause], outputs=output)
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# del X_word, X_pos, X_clause, Y_ner  # ลบตัวแปรที่ไม่ได้ใช้
# gc.collect()  # เรียก Garbage Collector

C:\Users\ThugCom\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ word_input          │ (None, 2081)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pos_input           │ (None, 2081)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ clause_input        │ (None, 2081)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 2081, 32)  │    897,152 │ word_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 2081, 8)   │        128 │ pos_input[0][0]   │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 2081, 4)   │         16 │ clause_input[0][… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 2081, 44)  │          0 │ embedding[0][0],  │
│ (Concatenate)       │                   │            │ embedding_1[0][0… │
│                     │                   │            │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 2081, 128) │     55,808 │ concatenate[0][0] │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 2081, 40)  │      5,160 │ bidirectional[0]… │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 958,264 (3.66 MB)

 Trainable params: 958,264 (3.66 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# ## 5. Train the Model

# history = model.fit(
#     [X_train_word, X_train_pos, X_train_clause], np.array(Y_train),
#     validation_data=([X_val_word, X_val_pos, X_val_clause], np.array(Y_val)),
#     batch_size=32,
#     epochs=10,
#     verbose=1
# )
history = model.fit(
    [X_word, X_pos, X_clause],  # อินพุต (เป็นลิสต์สำหรับหลายอินพุต)
    Y_ner,                     # เอาต์พุต (one-hot หรือ integer)
    batch_size=12,
    epochs=10,
    validation_split=0.2
)


# Fit the model
# history = model.fit([X_word, X_pos, X_clause], np.array(Y_ner), batch_size=32, epochs=10)

Epoch 1/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 759s 3s/step - accuracy: 0.8916 - loss: 0.7377 - val_accuracy: 0.9539 - val_loss: 0.1701
Epoch 2/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 731s 3s/step - accuracy: 0.9576 - loss: 0.1521 - val_accuracy: 0.9759 - val_loss: 0.0869
Epoch 3/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 701s 3s/step - accuracy: 0.9770 - loss: 0.0817 - val_accuracy: 0.9838 - val_loss: 0.0568
Epoch 4/10
 48/253 ━━━━━━━━━━━━━━━━━━━━ 9:24 3s/step - accuracy: 0.9827 - loss: 0.0598

In [ ]:
# ## 6. Prepare Test Data

test_word = test_data.groupby('line_id')['word'].apply(list).values
test_pos = test_data.groupby('line_id')['pos'].apply(list).values
test_clause = test_data.groupby('line_id')['clause_boundary'].apply(list).values

test_word = pad_sequences(test_word, maxlen=max_len, padding='post')
test_pos = pad_sequences(test_pos, maxlen=max_len, padding='post')
test_clause = pad_sequences(test_clause, maxlen=max_len, padding='post')


In [ ]:
# ## 7. Predict and Create Submission

predictions = model.predict([test_word, test_pos, test_clause])

# Convert predictions to class indices
predicted_classes = np.argmax(predictions, axis=-1)

# Generate submission
submission = []
for i, seq in enumerate(predicted_classes):
    for j, pred in enumerate(seq):
        submission.append({'index': f'test_{i}_{j}', 'name entity': pred})

submission_df = pd.DataFrame(submission)
submission_df.to_csv('submission.csv', index=False)

In [ ]:
submission_df